In [6]:
from __future__ import unicode_literals
import spacy
nlp = spacy.load('en_core_web_sm')
para = ''' The room fell silent when the TV newscaster reported the story of the earthquake.'''
doc = nlp(para)

# To extract sentences based on key word
indicated_for_sents = [sent for sent in doc.sents]
print(indicated_for_sents)
print()
# To extract objects of verbs
for word in doc:
    if word.dep_ in ('pobj'):
        subtree_span = doc[word.left_edge.i : word.right_edge.i + 1]
        print(subtree_span.text, word)

[ The room fell silent when the TV newscaster reported the story of the earthquake.]

   
the earthquake earthquake


In [97]:
import nltk
from nltk.tag import pos_tag
import re

statement = para
statement = "Thieves made off with Edvard Munch's, The Scream before police could stop them."
statement = "The dog was very happy because he just finished eating."
statement = "People were crying in the streets when they thought the sky was falling."
sentenceParts = pos_tag(statement.split()) 
print(sentenceParts)


def sentSegs(taggedSent, grammar, loops = 1):     # sentenceParts; grammar = "NP: {<DT>?<JJ>*<NN>}"
    #if loops == "":
    #    loops = "loop=1"
    cp = nltk.RegexpParser(grammar, loops)
    result = cp.parse(taggedSent)
    return result

grammar = r"""
    NP:
        {<.*>+}
        }<IN|CC>+{
"""   

grammar = r"""                                                              # Mary saw the cat sit on the mat will yield
  NP: {<DT|JJ|PRP|NN.*>+}          # Chunk sequences of DT, JJ, NN                (S
  PP: {<IN><NP><>}              # Chunk prepositions followed by NP              (NP Mary/NN)
  VP: {<MD>?<RB.*>?<VB.*><RB.*>?<NP|PP|RP|CLAUSE>+} # Chunk verbs and their arguments                saw/VBD
  CLAUSE: {<NP><VP>+}           # Chunk NP, VP                                   (CLAUSE
"""                 

out = sentSegs(sentenceParts, grammar, 2)

print(out)

[('People', 'NNS'), ('were', 'VBD'), ('crying', 'VBG'), ('in', 'IN'), ('the', 'DT'), ('streets', 'NNS'), ('when', 'WRB'), ('they', 'PRP'), ('thought', 'VBD'), ('the', 'DT'), ('sky', 'NN'), ('was', 'VBD'), ('falling.', 'JJ')]
(2
  (NP People/NNS)
  were/VBD
  crying/VBG
  in/IN
  (NP the/DT streets/NNS)
  when/WRB
  (CLAUSE
    (NP they/PRP)
    (VP thought/VBD (NP the/DT sky/NN))
    (VP was/VBD (NP falling./JJ))))


In [96]:

OWL_dependent_markers = set(["after", "although", "as", "as if", "because", "before", "even if", "even though", "if", "in order to", "since", "though", "unless", "until", "whatever", "when", "whenever", "whether", "while"])

new_str = []
clause_found = False
clause_at = -1
for enum, item in enumerate(out):
    if type(item) == nltk.Tree:
        if clause_found:
            for leaf in item.leaves():
                new_str.append(leaf[0])
        else:
            pass
    
    elif type(item) == tuple:
        if item[0] in OWL_dependent_markers and not clause_found:
            print("possible clause found")
            if out[enum + 1].label() == "CLAUSE":
                clause_found = True
                clause_at = enum
                new_str.append(item[0])
                print("clause found")
        
        elif clause_found:
            new_str.append(item[0])

if clause_found:
    for i in range(clause_at):
        if type(out[i]) == nltk.Tree:
            for leaf in out[i].leaves():
                new_str.append(leaf[0])
        elif type(out[i]) == tuple:  
            new_str.append(out[i][0])  

print(new_str)

possible clause found
clause found
['when', 'they', 'thought', 'the', 'sky', 'was', 'falling.', 'People', 'were', 'crying', 'in', 'the', 'streets']


In [ ]:
#https://www.reddit.com/r/LanguageTechnology/comments/b8xdb1/find_independent_clause_from_a_statement/
#https://owl.purdue.edu/owl/general_writing/academic_writing/sentence_variety/similar_sentence_patterns_or_rhythms.html
#https://owl.purdue.edu/owl/general_writing/punctuation/independent_and_dependent_clauses/index.html#:~:text=A%20dependent%20marker%20word%20is,it%20into%20a%20dependent%20clause.&text=Some%20common%20dependent%20markers%20are,whenever%2C%20whether%2C%20and%20while.
#https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk
